<a href="https://colab.research.google.com/github/abhishekguptait/ds_summit/blob/master/queen_ga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Science Summit Python Challenge**

*Description*

The eight queens puzzle is problem of placing eight queens on 8×8 chessboard so no two queens attack each other. A possible solution requires that no two queens share the same row, column, or diagonal. The eight queens puzzle is an example of the more general n queens problem of placing n non-attacking queens on an n×n chessboard. (Source : https://en.wikipedia.org/wiki/Eight_queens_puzzle )

*Challenge*

The challenge is to generate one right sequence through Genetic Programming. The sequence has to be 8 numbers between 0 to 7. Each number represents the positions the Queens can be placed. Each number refers to the row number in the specific column

`2 4 6 0 3 1 7 5`

· 2 is the row number in the column 0 where the Queen can be placed

· 4 is the row number in the column 1 where the Queen can be placed

This challenge can have 92 correct sequences. Anyone valid sequence generation is enough for our purpose. Program can be tweaked to generate more sequences.

In [10]:
import threading 
import sys
import _thread
import numpy as np
from numpy.random import choice
import pandas as pd
import time
import random


global valid_code,right_sequence
valid_code="0"

def main():
    return_list1=[]
    global valid_code
    global right_sequence
    global iteration
    
    ##### These varaibles can be tuned further as per assessment #####
    chessboard_size=8
    mutationPercentage = 0.01
    totalPopulation = 150
    crossOver = 0.5
    generationCount = 1000
    
    
    return_list1=NumberOfQueens(chessboard_size).solve() 

    for index in range(len(return_list1)):
            thread_name=''.join(return_list1[index])
            thread_instance = threading.Thread(target=preprocessing, args=(chessboard_size,thread_name,mutationPercentage,totalPopulation,crossOver,generationCount,)) 
            thread_instance.start()
    print("\n"+" # The "+ right_sequence +" is the valid first sequence generated in iteration :"+iteration+"#")
    
def inital_population_setup(totalPopulation,chessboard_size,target,secure_random,alpha_list):     
      populationData1 = []
      fitnessData1 = []
      for outloop in range(totalPopulation):
            randomData = []
            fitnessScore = 0
            for inloop in range(chessboard_size):
                     selectedData = secure_random.choice(alpha_list)
                     if (selectedData == target[inloop]):
                           fitnessScore = fitnessScore + 1
                     randomData.append(selectedData)
            populationData1.append(randomData)
            fitnessData1.append(fitnessScore)
      return (populationData1,fitnessData1)
      


def preprocessing(chessboard_size,thread_name,mutationPercentage,totalPopulation,crossOver,generationCount): 
      global valid_code,iteration
      
     
      
      secure_random = random.SystemRandom()
      target = thread_name
      alpha_list = [chr(x) for x in range(ord('0'), ord('7')+1 )]
      populationData = []
      fitnessData = []
      populationData,fitnessData=inital_population_setup(totalPopulation,chessboard_size,target,secure_random,alpha_list)
      probabilityDist = []
      for outloop in range(totalPopulation):
            probabilityDist.append(fitnessData[outloop]/len(target))
      probDataFrame = pd.DataFrame({'String':populationData,'FitnessScore':fitnessData,'Probability':probabilityDist})
      probDataFrame = probDataFrame.sort_values(['Probability'],ascending=False)
      probDataFrame = probDataFrame.reset_index(drop=True)
      probDataFrame.head()
      crossover_and_mutation(alpha_list,crossOver,target,generationCount,probDataFrame,thread_name,secure_random,populationData,probabilityDist,fitnessData)


        
        
def crossover_and_mutation(alpha_list,crossOver,target,generationCount,probDataFrame,thread_name,secure_random,populationData,probabilityDist,fitnessData):
      global valid_code,right_sequence,iteration

      crossOverPoint = int(crossOver*len(target))
      #print("\n")
      for loop in range(generationCount):
        if valid_code=="1":
              sys.exit(0)
        else:
              draw=[]
              draw.append(probDataFrame[0:1]["String"].values[0])
              draw.append(probDataFrame[1:2]["String"].values[0])
              if (getFitnessScore(draw[0],target)==len(target) | getFitnessScore(draw[1],target)==len(target)):
                    print_str=""
                    print_str0=''.join([elem for elem in draw[0]])
                    print_str1=''.join([elem for elem in draw[1]])
                    #print_str="Final score|draw[0]:"+print_str0+"|FS:"+str(getFitnessScore(draw[0],target))+"|draw[1]:"+print_str1+"|FS:"+str(getFitnessScore(draw[1],target))
                    #print_str="Final score|draw[0]:"+print_str0+"|FS:"+getFitnessScore(draw[0],target)
                    print(print_str)
                    
                    valid_code="1"
                    right_sequence=thread_name
                    
                    iteration=str(loop)
                    #print(probDataFrame.head(30))
                    sys.exit(0)
                    break
              child1 = draw[0][0:crossOverPoint]+draw[1][crossOverPoint:]
              child2 = draw[1][0:crossOverPoint]+draw[0][crossOverPoint:]
              child1[random.randint(0,len(target)-1)] = secure_random.choice(alpha_list)
              child2[random.randint(0,len(target)-1)] = secure_random.choice(alpha_list)
              populationData.append(child1)
              populationData.append(child2)
              fitnessData = []
              totalPopulation = len(populationData)
              for outloop in range(totalPopulation):
                    fitnessScore = getFitnessScore(populationData[outloop],target)
                    fitnessData.append(fitnessScore)
              probabilityDist = []
              for outloop in range(totalPopulation):
                    probabilityDist.append(fitnessData[outloop]/sum(fitnessData))
              probDataFrame = pd.DataFrame({'String':populationData,'FitnessScore':fitnessData,'Probability':probabilityDist})
              probDataFrame = probDataFrame.sort_values(['Probability'],ascending=False)
              probDataFrame = probDataFrame.reset_index(drop=True)
              if valid_code=="1":
                    sys.exit(0)
              else:
                    
                    a=""
                    #op_string="\n"+'Thread '+str(thread_name)+' Generation '+str(loop)+' '+' Average Fitness Score '+str(probDataFrame["FitnessScore"].mean())+' '+ ''.join(elem for elem in child1)+' '+str(getFitnessScore(child1,target))+' '+''.join(elem for elem in child2)+' '+str(getFitnessScore(child2,target))
                    op_string="\n"+'Thread '+str(thread_name)+' Generation '+str(loop)+' '+'|Child1 :'+ ''.join(elem for elem in child1)+'|FS of Child1: '+str(getFitnessScore(child1,target))+'|Child2 :'+''.join(elem for elem in child2)+'|FS of Child2 :'+str(getFitnessScore(child2,target))
                    print(op_string)
                    
            
        


    

def getFitnessScore(data,target):

          data = ''.join([elem for elem in data])
          fitnessScore = 0
          for inloop in range(len(target)):
                if (data[inloop] == target[inloop]):
                        fitnessScore = fitnessScore + 1
          #print ("data :",data)
          #print ("target :", target)
          #print ("fitness score :", fitnessScore)
          return fitnessScore


    

def maxProbability(probabilityDist):
        probabilityList = [f for f in set(probabilityDist)]
        return (probabilityList[len(probabilityList)-2])


def viewDataElement(data):
          data = ''.join([elem for elem in data])
          return data

        
        
class NumberOfQueens:
    """Class to generate all valid sequences solutions for the n queens puzzle through GA"""
    def __init__(self,size):
        # Store the size of Problem and number of valid solutions
        self.size = size
        self.solutions = 0


    def solve(self):
        """Method to determine solution of n queens puzzle and print the number of solutions"""

        final_list=[]
        positions = [-1] * self.size
        self.place_queen(positions, 0,final_list)
        #print("Found", self.solutions, "solutions.")
        return final_list


    def place_queen(self, positions, nextTargetRow,final_list):
        """
        Try to place a queen on nextTargetRow by checking all N possible cases.
        If a valid place is found the function calls itself trying to place a queen
        on the next row until all N queens are placed on the NxN board.
        """
        # Case - When all N rows are occupied Then Stop
        
        if nextTargetRow == self.size:
            
            self.show_full_chessboard(positions,final_list)
            # self.show_short_chessboard(positions)
            self.solutions += 1
        else:
            # For all N columns positions try to place a queen
            for column in range(self.size):
                # Reject all invalid positions
                if self.validate_place(positions, nextTargetRow, column):
                    positions[nextTargetRow] = column
                    self.place_queen(positions, nextTargetRow + 1,final_list)
        


    def validate_place(self, positions, ocuppied_rows, column):
        """
        Validate if the position is under threat from any of
        the previously placed queens (Validate column and diagonal side)
        """
        for i in range(ocuppied_rows):
            if positions[i] == column or \
                positions[i] - i == column - ocuppied_rows or \
                positions[i] + i == column + ocuppied_rows:

                return False
        return True

    def show_full_chessboard(self, positions,final_list):
        """Show the full NxN ChessBoard"""
        
        small_list=[]
        for row in range(self.size):
            line = ""
            for column in range(self.size):
                if positions[row] == column:
                    #line += "Q "
                    small_list.append(str(column))
                    #print("Pervez :{},{}".format(row,column))
                #else:
                    #line += ". "
                    
            #print(line)
        final_list.append(small_list)
        #print("\n")
        

    def show_short_chessboard(self, positions):
        """
        Show the queens positions on the board in compressed/short form,
        each number represent the occupied column position in the corresponding row.
        """
        line = ""
        for i in range(self.size):
            line += str(positions[i]) + " "
        #print(line)

if __name__ == "__main__":
    # execute only if run as a script

    main()



 # The 72051463 is the valid first sequence generated in iteration :55#
